In [ ]:
from gpt import GPT

In [ ]:
# make sure to put your openai key in 'data/.env' before creating GPT instance 
gpt_instance = GPT(model='gpt-4o',prompt='default')

In [ ]:
# test to make sure gpt is setup with a single request
print(gpt_instance.make_openai_request('Where are the 2024 Olympics?'))

In [ ]:
# running batched prompts in parallel
# using metadata
# note: batches here AND in gpt.py so that streaming can be integrated for large datasets. Not needed for smaller inputs

# sample prompts:
prompts = [{'text':'Where are the 2024 Olympics?', 'source':'dataset1'},
           {'text': 'Write me a haiku about trees', 'source':'dataset2'}]

batch = []
batch_size = 10
final_responses = []

for entry in prompts:
    prompt = {
        "text": entry['text'], # do not change - prompts must be stored with 'text' key
        "metadata": { # do not change - metadata must be stored with 'metadata' key
            "source": entry['source'] # change this to reflect the metadata you wish to track
        },
    }
    batch.append(prompt)

    if len(batch) >= batch_size:
        # get responses from GPT:
        responses = await gpt_instance.process_prompts_in_batches_async(batch)
        final_responses.extend(
            [
                {**prompt["metadata"], **response} # this is to make sure we can match prompts and responses back to their source
                for prompt, response in zip(batch, responses)
            ]
        )
        batch = []

    # run one more time to make sure nothing is left in final batch
    if batch:
        responses = await gpt_instance.process_prompts_in_batches_async(batch)
        final_responses.extend(
            [
                {**prompt["metadata"], **response}
                for prompt, response in zip(batch, responses)
            ]
        )

In [ ]:
#print GPT's responses:
print(final_responses)